In [60]:
#!/usr/bin/env sage
# Created on Nov 27, 2020
# author: Hosein Hadipour
# contact: hsn.hadipour@gmail.com
import sys
from graphviz import Digraph

def dtf_to_dg(file_name):
    with open(file_name, "r") as detfile:
        lns = detfile.readlines()
    output_buffer = ""
    DG_graphviz = Digraph(name='DataFlow', node_attr={'shape': 'circle'})
    known_variable_line_number = [i + 1 for i in range(len(lns)) if "are initially known" in lns[i]]
    if known_variable_line_number != []:
        known_variables = lns[known_variable_line_number[0]].split(", ")
        known_variables[-1] = known_variables[-1].replace("\n", "")
    initial_guessed_vars_line_number = [i + 1 for i in range(len(lns)) if "variable(s) are guessed:" in lns[i]]
    if initial_guessed_vars_line_number != []:
        initial_guessed_vars = lns[initial_guessed_vars_line_number[0]].split(", ")
        initial_guessed_vars[-1] = initial_guessed_vars[-1].replace("\n", "")
    vertices = []
    edges = []
    cnt = 0
    for ln in lns:
        if "===>" in ln:
            if ": " in ln:
                temp = ln.split(": ")[1][:-1]
            else:
                temp = ln
            temp = temp.split(" ===> ")
            rhs = temp[1]
            lhs_temp = temp[0].split(", ")
            lhs = []
            for el in lhs_temp:
                el = el.replace(" ", "")
                vertices.append(el)
                edges.append((el, rhs))
                node1 = el.split("_")
                node1 = "{%s_{%s}}" % (node1[0], ",".join(node1[1:]))
                node2 = rhs.split("_")
                node2 = "{%s_{%s}}" % (node2[0], ",".join(node2[1:]))
                DG_graphviz.edge(node1, node2)
            cnt += 1
    # Symbolize vertices names
    vertices = list(map(var, vertices))
    if initial_guessed_vars_line_number != []:
        initial_guessed_vars = list(map(var, initial_guessed_vars))
        vertices = list(set(vertices + initial_guessed_vars))
    else:
        vertices = list(set(vertices))
    # Symbolize edges' vertiices
    for i in range(len(edges)):
        edges[i] = (var(edges[i][0]), var(edges[i][1]))
    # Symbolize known_variables
    if known_variable_line_number != [] and known_variables != [""]:
        known_variables = list(map(var, known_variables))
    DG = DiGraph([vertices, edges])
    DG_sources = DG.sources()
    for v in DG.vertices():
        if v in DG_sources:
            node = str(v).split("_")
            node = "{%s_{%s}}" % (node[0], ",".join(node[1:]))
            DG_graphviz.node(node, color="red", style="filled", shape="doublecircle", texmode="math")
        else:
            node = str(v).split("_")
            node = "{%s_{%s}}" % (node[0], ",".join(node[1:]))
            DG_graphviz.node(node, color="green", style="filled", texmode="math")
    return vertices, edges, DG, DG_graphviz

In [61]:
V, E, DG, DG_graphviz = dtf_to_dg(file_name="output.txt")
# LC = DG.connected_components_subgraphs()
# size_of_source = lambda x: len(x.sources())
# MSG = DiGraph()
# cnt = 0
# for sg in LC:
#     print("Sub-graph %d: %s, Root size: %d, Number of vertices: %d" % (cnt, sg, len(sg.sources()), len(sg.vertices())))
#     cnt += 1
#     new_size = size_of_source(sg)
#     current_size = size_of_source(MSG)
#     if current_size < new_size:
#         MSG = sg
# guessed_vars = MSG.sources()
# for kv in known_variables:
#     if kv in guessed_vars:
#         guessed_vars.remove(kv)
# print("Number of guessed variables: %d" % len(guessed_vars))
# print("Guessed variables: ")
# print(guessed_vars)
#MSG.plot()

In [62]:
DG

Digraph on 481 vertices (use the .plot() method to plot)

In [64]:
# DG_graphviz.node_attr["fontsize"] = "10"
# DG_graphviz.node_attr["fixedsize"] = "true"
DG_graphviz.graph_attr["layout"] = "dot"
DG_graphviz.graph_attr["splines"] = "true"
# DG_graphviz.graph_attr["epsilon"] = 

In [66]:
import dot2tex
texcode = dot2tex.dot2tex(DG_graphviz.source, format='tikz', crop=True)
print(texcode)

\documentclass{article}
\usepackage[x11names, svgnames, rgb]{xcolor}
\usepackage[utf8]{inputenc}
\usepackage{tikz}
\usetikzlibrary{snakes,arrows,shapes}
\usepackage{amsmath}
%
%

\usepackage[active,tightpage]{preview}
\PreviewEnvironment{tikzpicture}
\setlength\PreviewBorder{0pt}%

%

\begin{document}
\pagestyle{empty}
%
%
%

\enlargethispage{100cm}
% Start of code
\begin{tikzpicture}[>=latex',line join=bevel,]
%%
\node ({k_{17+37}}) at (201.19bp,963.53bp) [draw=red,fill=red,circle, double] {${k_{17,37}}$};
  \node ({k_{18+8}}) at (203.19bp,809.14bp) [draw=green,fill=green,circle] {${k_{18,8}}$};
  \node ({k_{19+59}}) at (207.19bp,658.75bp) [draw=green,fill=green,circle] {${k_{19,59}}$};
  \node ({k_{17+38}}) at (341.19bp,963.53bp) [draw=red,fill=red,circle, double] {${k_{17,38}}$};
  \node ({k_{18+9}}) at (340.19bp,809.14bp) [draw=green,fill=green,circle] {${k_{18,9}}$};
  \node ({k_{19+60}}) at (340.19bp,658.75bp) [draw=green,fill=green,circle] {${k_{19,60}}$};
  \node ({k_{17+39}}) 

In [496]:
# for G in LC:
# #     G.plot()
#     G.show(vertex_labels=True, edge_labels=False, vertex_size=0, vertex_shape="o" , 
#            talk=True, edge_style="-", layout="circular")